# Python fundamentals for AI (notater)

Dette er ryggmargs-notater for **Python → ML/NLP → LLM**, med små kodeeksempler du kan kjøre.

## Innhold

- 1) Variabler og datatyper
- 2) Operatorer
- 3) If/else
- 4) Funksjoner
- 5) Sekvenser: list/tuple/dict
- 6) Løkker: for/while + `range`
- 7) Mental bro: Python → ML/NLP → LLM
- 8) Backdoors i LLMs + metrikker (ASR/CACC)
- 9) Post-training defense: Model Merge
- 10) Mini “bachelor-cheat sheet” + vanlige feil


## 1) Variabler og datatyper (byggesteiner)

- `int` (heltall), `float` (desimaltall), `str` (tekst), `bool` (True/False)
- I AI-kode: dette brukes til **konfig**, **hyperparametre**, **labels**, **tekster**, **flag**.

**Eksempel (ekte verden):**

- Du lager en spam-detektor: `label = 1` betyr spam, `0` betyr ikke-spam.


In [ ]:
# Variabler og datatyper
lr = 2e-5          # float (learning rate)
epochs = 3         # int
text = "Hello..."  # str
is_train = True    # bool

label = 1          # 1 = spam, 0 = ikke-spam

type(lr), type(epochs), type(text), type(is_train), type(label)

## 2) Operatorer (regning + logikk)

- **Matematikk**: `+ - * / **`
- **Sammenligning**: `== != < > <= >=`
- **Logikk**: `and`, `or`, `not`
- **Modulo**: `%` (superviktig i odd/even, batching, etc.)


In [ ]:
x = 7
if x % 2 == 1:
    print("odd")
else:
    print("even")

## 3) If/else (beslutning)

Brukes hele tiden i datasett-filtrering: “ta bare samples som matcher kriteriet”.


In [ ]:
price = 6
qty = 2
money_spent = 0

if price >= 5:
    money_spent += price * qty

money_spent

## 4) Funksjoner (gjenbrukbar logikk)

Tenk: “mini-program” du kan teste isolert.

**Rektangel-oppgaven (kjerneidé):**

- Returnerer **to verdier** → i Python blir det en **tuple** automatisk.


In [ ]:
def rectangle_info(length, width):
    area = length * width
    perimeter = 2 * (length + width)
    return area, perimeter

area, perimeter = rectangle_info(2, 10)
area, perimeter

## 5) Sekvenser (list, tuple, dict) + “hvordan tenke riktig”

### Liste `[]` (ordna, kan endres)

Brukes i AI til: batches, tokens, scores, logger.

### Tuple `()` (ordna, “låst”)

Brukes til: returnere flere ting (`(loss, acc)`), koordinater, config.

### Dictionary `{key: value}` (oppslag med nøkkel)

Dette er “database-følelse”: du finner verdi med **nøkkel**, ikke index.

**Viktig læring fra `Price_list`-feilen:**

- Testen forventet at **keys er selve måltidsnavnene** (f.eks. `"Spaghetti"`), ikke `"meal_1"`.
- Mønsteret er veldig vanlig: **bruk verdien fra én dict som nøkkel i en annen**.


In [ ]:
nums = [1, 2, 3]
nums.append(4)

cars = ("BMW", "Dodge", "Ford")

menu = {"meal_1": "Spaghetti", "meal_2": "Fries"}
price_list = {
    # key = meal name (value from menu), value = price
    menu["meal_1"]: 12.0,
    menu["meal_2"]: 5.0,
}

nums, cars, menu, price_list, price_list["Fries"]

## 6) Løkker og `range` (motoren i alt som er “trening/evaluering”)

### `range(start, stop, step)` (stop er eksklusiv)

- `range(0, 20)` gir 0…19
- `range(0, 31, 2)` gir partall 0…30

Dette er sentralt i batching og epoch-loops.


In [ ]:
# For-loop: odd/even printing
out = []
for x in range(1, 31):
    if x % 2 != 0:
        out.append(str(x))
    else:
        out.append("Even")

" ".join(out[:20]) + " ..."

### While-loop (når du vil styre stopp selv)

Typisk i AI: “kjør til ingen forbedring” / “kutt når condition slår inn”.

**Viktig:** `return` skal stå **etter** løkka (ellers returnerer du for tidlig).


In [ ]:
def count_smaller_than(nums, threshold=20):
    i = 0
    counter = 0
    while i < len(nums):
        if nums[i] < threshold:
            counter += 1
        i += 1
    return counter

count_smaller_than([3, 25, 19, 100, 0, 20, 18], threshold=20)

## 7) “Python → ML/NLP → LLM” (den mentale broa)

### Hvordan en ML/NLP-pipeline ofte ser ut

1. Data inn (CSV/JSON/text)
2. Preprocessing (cleaning, tokenisering)
3. Split (train/val/test)
4. Train loop (forward → loss → backward → update)
5. Eval (accuracy/F1, etc.)
6. Security-testing (trigger-tester / robusthet)

Samme struktur enten du gjør klassisk ML eller Transformers.

### Hugging Face + PyTorch (hva du må forstå)

- batching
- `attention_mask`
- `labels`
- forskjell på `model.train()` og `model.eval()`


## 8) Backdoor i LLMs (kjerne for bachelorprosjekt)

### Hva er en backdoor?

En backdoored modell oppfører seg normalt på “rene” inputs, men når en **trigger** dukker opp, blir output tvunget mot et “malicious” mål.

- Ren input: \(x \rightarrow y\)
- Med trigger/poisoning: \(x' \rightarrow y'\)

### To hovedmåter

1. **Data poisoning**: trenings-eksempler endres til \((x', y')\) med trigger + target label.
2. **Weight poisoning / post-training**: angriper manipulerer vekter etter trening.

### Hvordan måler man det?

- **CACC** = Clean Accuracy (ytelse på ren test)
- **ASR** = Attack Success Rate (hvor ofte triggeren “får viljen sin”)

Målet i forsvar: **ASR ned** uten at **CACC faller mye**.


## 9) Post-training defense: Model Merge (paper)

Paperet “Here’s a Free Lunch: Sanitizing Backdoored Models with Model Merge” (ACL 2024) foreslår en **inference-stage defense**: ingen retrening, du **merger** en mistenkt modell med andre modeller (samme arkitektur/oppgave).

En enkel variant er weight averaging:

\[
W' = \frac{W_p + \sum_k W_k}{n}
\]

**Intuisjon:** backdoor-signalet blir “minority” når du blander flere modeller → trigger-effekten blir svakere.

**Begrensninger:**

- typisk samme arkitektur
- samme type output/oppgave


## 10) Mini “bachelor-cheat sheet”

### Datasett-struktur (typisk)

Hver rad:

- `text`
- `label` (clean)
- `poisoned` (0/1) eller `trigger_type`
- `target_label` (hvis relevant)

### Trigger-testing (enkelt men effektivt)

Test samme input:

- uten trigger
- med trigger

…og mål om output flipper mot target.

### Logg alltid (sensor-vennlig)

- dato, modellnavn, seed, hyperparametre
- ASR, CACC
- hvilke triggere du testet

## 11) Vanlige student-feil (du har allerede møtt flere)

1. Dict: keys ≠ values (`Menu["meal_2"]` vs `"Fries"`)
2. `range`: stop er eksklusiv
3. `return` inne i løkke → stopper for tidlig
4. `get()` uten key (`Price_list.get()`)
5. feil formel: `perimeter = 2*(length*width)` (skal være `2*(length+width)`)
6. i løkker: skriver `n * 10` uten å bruke resultatet (må `print(n*10)` eller lagre)


---

# Utvidelser (mer dybde + flere AI-relevante eksempler)

Nedenfor bygger vi videre på hvert punkt med litt mer intuisjon, vanlige fallgruver, og små mønstre du faktisk bruker i ML/NLP.


## 1) Variabler og datatyper — mer dybde

### A) Konvertering (casting) og parsing

I AI-prosjekter kommer config ofte inn som tekst (CLI/JSON/YAML). Da må du konvertere trygt:

- `int("3")` → `3`
- `float("2e-5")` → `2e-05`
- `bool("False")` funker **ikke** som du tror (alle ikke-tomme strenger er `True` i Python).

### B) Truthiness (supervanlig i filtering)

I `if x:` tolkes mange typer som sann/usann:

- `0`, `0.0`, `""`, `[]`, `{}`, `None` → **False**
- alt annet → **True**

Dette er nyttig, men kan gi bugs hvis du blander “mangler verdi” og “verdien er 0”.

### C) `None` = “mangler”

Bruk `None` for “ukjent/ikke satt” (f.eks. `seed=None`), og test med `is None`.


In [ ]:
# Casting + truthiness

epochs_str = "3"
lr_str = "2e-5"

epochs = int(epochs_str)
lr = float(lr_str)

# Truthiness
values = [0, 1, 0.0, 0.1, "", "hi", [], [1], {}, {"a": 1}, None]
truth = [(v, bool(v)) for v in values]

epochs, lr, truth

## 2) Operatorer — mer dybde

### A) Precedence (rekkefølge)

`**` (potens) skjer før `*` og `/`, som skjer før `+` og `-`.

I AI-kode er dette viktig i f.eks. normalisering og temperatur-scaling, så bruk parenteser for å være tydelig.

### B) Flyttall (float) og `==`

Ikke stol på `==` med floats pga. avrunding. Bruk heller en toleranse:

- `abs(a - b) < 1e-9`

### C) Korte “guard conditions”

Mange bugs forebygges med en tidlig sjekk:

- `if batch_size <= 0: raise ValueError(...)`


In [ ]:
# Float-sammenligning

a = 0.1 + 0.2
b = 0.3

exact = (a == b)
approx = abs(a - b) < 1e-12

a, b, exact, approx

## 4) Funksjoner — mer dybde (lesbarhet = fart)

### A) Docstring + type hints

Når bachelor-koden blir stor, blir dette gull. Det gjør:

- debugging lettere
- samarbeid enklere
- “hva forventer funksjonen?” tydelig

### B) Pure functions vs. side effects

- **Pure**: tar inn data → returnerer resultat (lett å teste)
- **Side effect**: printer, skriver fil, endrer global state (kan være riktig, men vanskeligere å teste)

I ML-pipelines prøver du ofte å gjøre preprocessing-funksjoner **pure**.


In [ ]:
from typing import Tuple


def rectangle_info_typed(length: float, width: float) -> Tuple[float, float]:
    """Return (area, perimeter) for a rectangle."""
    if length < 0 or width < 0:
        raise ValueError("length/width must be non-negative")

    area = length * width
    perimeter = 2 * (length + width)
    return area, perimeter


rectangle_info_typed(2, 10)

## 5) List/tuple/dict — mer dybde

### A) Mutability (hva kan endres?)

- `list` og `dict` er **mutable** (endringer skjer “in-place”)
- `tuple` og `str` er **immutable**

Dette betyr at hvis du sender en liste inn i en funksjon og endrer den, så er den endret utenfor også.

### B) `dict.get()` og “missing keys”

- `d[k]` krasjer med `KeyError` hvis `k` mangler
- `d.get(k)` gir `None` (eller default) i stedet

I datarensing er `get()` ofte bedre.

### C) Vanlige dict-mønstre i ML

- counters (f.eks. token-frekvens)
- mapping label → id (og id → label)
- config dict (hyperparametre)


In [ ]:
# Mutability + dict.get

def add_item_in_place(xs: list[int], x: int) -> None:
    xs.append(x)


a = [1, 2]
add_item_in_place(a, 3)

# get() vs []
label2id = {"ham": 0, "spam": 1}
maybe = label2id.get("unknown", -1)

# enkel token-counter
text = "hello hello ai"
tokens = text.split()
counts = {}
for t in tokens:
    counts[t] = counts.get(t, 0) + 1

a, maybe, counts

## 6) Løkker — mer dybde

### A) `enumerate` (når du trenger index + verdi)

I datasett-iterasjon er dette standard:

- `for i, x in enumerate(xs): ...`

### B) `zip` (parallel iterasjon)

Veldig vanlig for `(text, label)` eller `(pred, gold)`.

### C) List comprehensions (kort, men bruk med måte)

Kjempepraktisk i preprocessing:

- `clean = [x.strip() for x in lines if x.strip()]`

### D) “batching”-mønsteret du kommer til å bruke mye

Når du vil dele en liste i biter:

- `for start in range(0, n, batch_size): batch = xs[start:start+batch_size]`


In [ ]:
# enumerate, zip, comprehensions, batching

xs = [10, 20, 30]
indexed = [(i, x) for i, x in enumerate(xs)]

texts = ["hi", "buy now", "hello"]
labels = [0, 1, 0]
pairs = list(zip(texts, labels))

lines = ["  a ", "", " b", "   ", "c "]
clean = [x.strip() for x in lines if x.strip()]

# batching
batch_size = 2
batches = []
for start in range(0, len(texts), batch_size):
    batches.append(texts[start : start + batch_size])

indexed, pairs, clean, batches

## 7) ML-treningsløkke — “minimal mental modell”

Under er en enkel pseudostruktur (uten PyTorch-imports) som matcher det som skjer i `Trainer` også.

**Nøkkelidé:** treningsløkka er bare en loop + noen tall (loss/gradients) + en oppdatering.

1. `model.train()`
2. for batch i dataloader:
   - forward → `logits`
   - loss
   - backward
   - optimizer step
   - zero grad
3. `model.eval()` + eval-loop uten gradients

Hvis du skjønner dette, skjønner du 80% av “hva som skjer” i praksis.


## 8) ASR/CACC — litt mer presist

### CACC (Clean Accuracy)

Andel riktige prediksjoner på **ren** test:

\[
\mathrm{CACC} = \frac{\#\{i : \hat{y}_i = y_i\}}{N}
\]

### ASR (Attack Success Rate)

For en **targeted** backdoor (målet er en spesifikk `target_label`):

\[
\mathrm{ASR} = \frac{\#\{i : \hat{y}(x'_i) = y_{target}\}}{N_{trigger}}
\]

**Viktig i rapport:**

- Definer *hvilke inputs* du måler ASR på (alle? bare de som ikke allerede var target?)
- Definer *hvilken trigger* (type, plassering, frekvens)
- Rapporter både ASR og CACC (trade-off)


## 9) Model merge — hva du kan skrive i bachelor-teksten

### A) Hva betyr “merge” praktisk?

Du tar flere modeller med samme arkitektur (samme parameter-”shape”) og lager en ny modell ved å kombinere vekter.

- **Weight averaging (WAvg/WAG)**: gjennomsnitt per parameter
- Andre merges kan bruke vekting per lag, normalisering, eller “adapter”-baserte kombinasjoner

### B) Hvorfor kan det redusere backdoor?

En backdoor kan ses som et mønster i vektrommet som gjør at triggeren får stor effekt. Når du gjennomsnittliggjør med flere “rene” modeller, blir bidraget fra backdooren relativt mindre.

### C) Når kan det feile?

- modeller er trent på litt ulike oppgaver/domener → merge kan skade CACC
- arkitektur mismatch → umulig
- hvis angriperen har **sterk** backdoor eller flere modeller er kompromittert

### D) Hvordan du kan evaluere merge-forsvaret (enkelt oppsett)

1. Mål CACC/ASR på den mistenkte modellen
2. Merge med 1–k andre modeller
3. Mål CACC/ASR igjen
4. Plot ASR vs CACC for ulike k


## 11) Mini-øvelser (med fasit)

### Øvelse A — dict mapping

Du har:

- `menu = {"meal_1": "Spaghetti", "meal_2": "Fries"}`
- `prices = {"Spaghetti": 12.0, "Fries": 5.0}`

Lag en liste `costs` som gir prisen for hver `meal_*` i rekkefølge `meal_1`, `meal_2`.

### Øvelse B — batching

Gitt `texts = ["a", "b", "c", "d", "e"]` og `batch_size=2`, lag batches.

### Øvelse C — return utenfor loop

Skriv en funksjon som teller hvor mange tall som er negative i en liste.


In [ ]:
# Fasit

# Øvelse A
menu = {"meal_1": "Spaghetti", "meal_2": "Fries"}
prices = {"Spaghetti": 12.0, "Fries": 5.0}

costs = [prices[menu[k]] for k in ["meal_1", "meal_2"]]

# Øvelse B
texts = ["a", "b", "c", "d", "e"]
batch_size = 2
batches = [texts[i : i + batch_size] for i in range(0, len(texts), batch_size)]

# Øvelse C
def count_negative(xs: list[int]) -> int:
    c = 0
    for x in xs:
        if x < 0:
            c += 1
    return c

costs, batches, count_negative([1, -1, -2, 5, 0, -10])